In [ ]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import multiprocessing as mp
from datetime import datetime as dtm
from typing import Optional, Sequence

from darts.metrics import mape, mae

from darts.utils.statistics import check_seasonality
from darts.utils.statistics import plot_acf
from darts.utils.statistics import plot_pacf

from darts.models.forecasting.varima import VARIMA
from darts.models.forecasting.arima import ARIMA

from darts.timeseries import TimeSeries as TS
from sklearn.model_selection import ParameterGrid as PG

In [ ]:
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (SMAPE) between two arrays.

    Parameters:
        y_true (array-like): Array of true values.
        y_pred (array-like): Array of predicted values.

    Returns:
        float: SMAPE value.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    
    return np.mean(numerator / denominator) * 100

def load_training_data(market_name):
    #df = pd.read_csv('/home/zqiao/data_flake/imputed data/{}_data.csv'.format(market_name), index_col=0)
    df = pd.read_csv('/home/zqiao/data_flake/imputed data/pho_t_data.csv',index_col=0)
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    return df

def train_auto_varima_mp(
    X_train,
    Y_train,
    X_test,
    Y_test,
    min_d,
    max_d,
    trends,
    max_p,
    min_p,
    max_q,
    min_q,
    ntest
):
    if trends is None:
        trends = [None]

    param_vals = {
        "p": [i for i in range(min_p, max_p + 1)],
        "d": [i for i in range(min_d, max_d + 1)],
        "q": [i for i in range(min_q, max_q + 1)],
        "trend": trends,
    }
    param_grid = list(PG(param_vals))
    num_params = len(param_grid)

    results = {}
    pool = mp.Pool(processes=mp.cpu_count())
    for idx, params in enumerate(param_grid):
        print(f"training model {idx}/{num_params-1}: {params}")

        result = pool.apply_async(
            run_varima_experiment,
            kwds={
                "X_train": X_train,
                "Y_train": Y_train,
                "X_test": X_test,
                "Y_test": Y_test,
                "params": params,
                "ntest": ntest
            },
        )
        params = str(params)
        results[params] = result


    pool.close()
    pool.join()

    for key, value in results.items():
        if value.get() is not None:
            results[key] = value.get()
    else:
            results[key] = 0
    res_df = pd.DataFrame.from_dict(results, orient='index', columns=['metric_val'])
    res_df = res_df.sort_values('metric_val', ascending=True)
    best_params = res_df.index.values[1]
    best_params = ast.literal_eval(best_params)
    p = best_params['p']
    d = best_params['d']
    q = best_params['q']
    trend = best_params['trend']


    return p,d,q,trend, res_df



def run_varima_experiment(X_train, X_test, Y_train, Y_test, params, ntest):
    
    model = train_varima_model(X_train=X_train, Y_train=Y_train, **params)
    metric_val = evaluate_varima_model(model, X_test, Y_test, ntest)

    return metric_val

def train_varima_model(X_train, Y_train, p, d, q, trend):
    
    if d not in [0, 1]:
        raise ValueError(f"d can only take on values 0 or 1")

    model = ARIMA(p=p, d=d, q=q, trend=trend)
    model.fit(Y_train, future_covariates=X_train)
    
    return model

def evaluate_varima_model(model, 
                          X_test, 
                          Y_test, 
                          ntest):

    pred = model.predict(ntest, future_covariates=X_test)
    pred_transfer = pred.pd_dataframe()
    Y_test_transfer = Y_test.pd_dataframe()
    pred_transfer['real_hedonic_rent_submarket'][0] = Y_test_transfer['real_hedonic_rent_submarket'][0]
    pred_transfer['real_hedonic_rent_submarket'] = pred_transfer['real_hedonic_rent_submarket'].cumsum()
    pred_new = TS.from_dataframe(pred_transfer)
    
    Y_test_series = Y_test_transfer['real_hedonic_rent_submarket']
    pred_series = pred_transfer['real_hedonic_rent_submarket']
    err = smape(Y_test_series, pred_series)

    return err


def run_varima_pipeline(market_name: str = None,
                        submkt_id: Optional[Sequence[str]] = None, 
                        target = "real_hedonic_rent_submarket",
                        features: list = None,
                        target_rolling: bool = None,
                        a_shift: bool = None,
                        ntest: int = None,
                        nlag: int = None,
                        rol_num: int = None,
                        min_d: int = None,
                        max_d: int = None,
                        trends: Optional[Sequence[str]] = None,
                        max_p: int = None,
                        min_p: int = None,
                        max_q: int = None,
                        min_q: int = None):
    
    if market_name is None:
        market_name = 'pho'
    
    if submkt_id is None:
        submkt_id = 'PHO037'

    df = load_training_data(market_name)

    grouped_df = df.groupby('research_submkt_id')
    for submkt, submkt_group in grouped_df:
        if submkt == submkt_id:
            submkt_df = submkt_group
    
    if ntest is None:
        ntest = 12
    
    if nlag is None:
        nlag = 6
        
    if features is None:
        features = [
            "real_market_level_rent",
            "gdp_histfc",
            "nominal_retail_sales_histfc",
            "employment_histfc",
            "real_ecommerce",
            "spread_3m10y",
            "real_retail_sales_ex_gas",
            "imports_us",
            "ecomm^2_pop",
            "weighted_pop_estimate_cryr",
            "weighted_hh_estimate_cryr"]
    
    pdf = submkt_df[[target] + features].copy()
    pdf["real_market_level_rent"] = pdf["real_market_level_rent"].diff()
    if a_shift:
        #pdf["avrate"] = pdf["avrate"].shift(nlag)
        #pdf["real_market_level_rent"] = pdf["real_market_level_rent"].shift(nlag)
        for col in features:
            pdf[col] = pdf[col].shift(nlag)

    if target_rolling:
        pdf[target] = pdf[target].rolling(rol_num).mean()
   
    pdf = pdf.dropna()
    X = TS.from_dataframe(pdf[features])
    Y = TS.from_dataframe(pdf[[target]])
    X_train, X_test = X[:-ntest], X[-ntest:]
    Y_train, Y_test = Y[:-ntest], Y[-ntest:]
    
    p,d,q,trend,res_df = train_auto_varima_mp(
        X_train = X_train,
        Y_train = Y_train,
        X_test = X_test,
        Y_test = Y_test,
        min_d = min_d,
        max_d = max_d,
        trends = trends,
        max_p = max_p,
        min_p = min_p,
        max_q = max_q,
        min_q = min_q,
        ntest = ntest)

    print('This is the best params of varima model: p: ',p,', d: ',d,' q: ',q,' trend: ',trend )
    
    

    return X, Y, X_train, Y_train, X_test, Y_test, p, d, q, trend, res_df

In [ ]:
X, Y, X_train, Y_train, X_test, Y_test,p, d, q, trend, res_df = run_varima_pipeline(market_name = 'pho',
                                                       submkt_id = 'PHO038', 
                                                       target = 'real_hedonic_rent_submarket',
                                                       features = None,
                                                       target_rolling = False,
                                                       a_shift = False,
                                                       ntest = 24,
                                                       nlag = 3,
                                                       rol_num = 6,                      
                                                       min_d = 0,
                                                       max_d = 0,
                                                       trends = [None,'ct','t'],
                                                       max_p = 5,
                                                       min_q = 1,
                                                       max_q = 5,
                                                       min_p = 1)

In [ ]:
plot_pacf(Y, m=None, max_lag=24, alpha=0.01, fig_size=(10, 5))

In [ ]:
plot_pacf(Y, m=None, max_lag=24, alpha=0.01, fig_size=(10, 5))

In [ ]:
res_df

In [ ]:
final_model = train_varima_model(X_train, Y_train, p, d, q, trend)
Y_pred = final_model.predict(24, future_covariates=X_test)

In [ ]:


Y_pred.plot(label='pred')
Y_test.plot(label='test')